In [ ]:
import wandb
import pandas as pd
import matplotlib.pyplot as plt

# Replace these with your W&B project and group details
WANDB_PROJECT = "Hypergradient-RL"  # e.g., "my-reinforcement-learning-project"
WANDB_ENTITY = "rl-power"  # e.g., "my-username"
GROUP_NAME = "your-group-name"  # e.g., "experiment-1"

# Initialize W&B API
api = wandb.Api()

# Fetch all runs in the project
all_runs = api.runs(f"{WANDB_ENTITY}/{WANDB_PROJECT}")

# Filter runs by group name
group_runs = [run for run in all_runs if run.group == GROUP_NAME]

# Plot metrics for all runs in the group
metrics_to_plot = ["reward", "critic_loss"]  # Replace with your logged metric names

for run in group_runs:
    history = run.history()  # Fetch metrics for each run
    df = pd.DataFrame(history)
    
    # Plot each metric in metrics_to_plot
    for metric in metrics_to_plot:
        if metric in df.columns:
            plt.plot(df["_step"], df[metric], label=f"{run.name} - {metric}")

# Customize and show the plot
plt.xlabel("Steps")
plt.ylabel("Metrics")
plt.title(f"Metrics for Group: {GROUP_NAME}")
plt.legend()
plt.grid(True)
plt.show()
